In [37]:
from keras.applications import EfficientNetB7
from keras.datasets.cifar10 import load_data
import tensorflow as tf
from keras.models import Model, Sequential
import numpy as np
from keras.layers import Dense, Input, Dropout, UpSampling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from zipfile import ZipFile

# Download Cats and Dogs Dataset

In [38]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O cats_and_dogs.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


# Extract zip file

In [39]:
zip_ref = ZipFile('cats_and_dogs.zip', 'r')
zip_ref.extractall('cats_and_dogs')
zip_ref.close()

In [40]:
train_generator = ImageDataGenerator(rescale=1.0/255.0)
test_generator = ImageDataGenerator(rescale=1.0/255.0)


In [41]:
train_path = 'cats_and_dogs/cats_and_dogs_filtered/train'
test_path = 'cats_and_dogs/cats_and_dogs_filtered/validation'

In [42]:
data_train = train_generator.flow_from_directory(train_path, class_mode='binary', target_size=(150, 150), subset='training')
data_test = train_generator.flow_from_directory(test_path, class_mode='binary', target_size=(150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [44]:
ENet = EfficientNetB7(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

In [45]:
ENet.trainable = False

In [46]:
mlp = Sequential()
mlp.add(Flatten())
mlp.add(Dense(1024, activation=tf.keras.layers.LeakyReLU(0.01)))
mlp.add(Dropout(0.2))
mlp.add(Dense(256, activation=tf.keras.layers.LeakyReLU(0.01)))
mlp.add(Dense(1, activation='sigmoid'))

In [47]:
model = Sequential([ENet, mlp])

In [48]:
model.compile(optimizer='adam', loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [49]:
model.fit(data_train, epochs=2)

Epoch 1/2
63/63 [==============================] - 263s 4s/step - loss: 10.1186 - accuracy: 0.5045
Epoch 2/2
63/63 [==============================] - 236s 4s/step - loss: 1.8400 - accuracy: 0.5055


In [50]:
model.evaluate(data_test)

32/32 [==============================] - 107s 3s/step - loss: 0.8625 - accuracy: 0.5000


[0.8625397086143494, 0.5]